# Donders MML: Video Clipping and Audio-Video Alignment
![alt text](<Donders MML LOGO.png>)

### Info Documents 
Location Repository
Github Repository 
Jupyter Notebook


### Requirements
Please install the necessary packages in requirements.txt using pip install -r requirements.txt

## 0. Importing Relevant Packages 

In [16]:
import os             # Importing the os module which provides functions for interacting with the operating system
import pyxdf          # Importing pyxdf, a Python library for reading XDF files
import glob           # Importing the glob module which helps in finding files/directories with specific patterns
import pandas as pd   # Importing pandas library (abbreviated as pd), which is used for data manipulation and analysis
import numpy as np    # Importing numpy library (abbreviated as np), which is used for numerical computations
import wave           # Importing wave module for reading and writing WAV files (usually audio files) 
import struct         # Importing struct module which provides functions to convert between Python values and C structs
import math           # Importing math module which provides mathematical functions
import random         # Importing random module for generating random numbers
from scipy.io import wavfile  # Importing wavfile module from scipy.io (a library built on numpy), for reading and writing WAV files
import noisereduce as nr      # Importing noisereduce module for noise reduction in audio signals
import json            # Importing json module for working with JSON data
import cv2            # Importing OpenCV library for computer vision tasks
from moviepy.editor import (                # Importing various classes and functions from moviepy.editor module
                            VideoFileClip,  # Class for working with video files
                            AudioFileClip,  # Class for working with audio files
                            CompositeAudioClip)  # Class for composing audio clip
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip  # video  clipping fucntion 
from moviepy.video.io.VideoFileClip import VideoFileClip          # alternative video clipping function
import matplotlib.pyplot as plt                                   # Importing pyplot library to create figures and plot data 
from matplotlib.widgets import Slider  
import tkinter                                                    # GUI toolkit to open and save files
from tkinter import filedialog                                    # GUI toolkit to open and save files
import subprocess 
from tqdm.notebook import tqdm
import re                                                         # Importing re module for working with regular expressions


print("Everything was imported succesfully") #as terminal

Everything was imported succesfully


## 1. Establish Relevant Paths, Variabls & Functions 

In [35]:
# ------------ PATHS -----------------------------------------------------
input_video_folder  = './video_raw/'                             # this folder should only contain the videos you want to process. 
input_file_folder   = './data_processed/T1/marker_MULTIPLEpairs'    # this folder contains the csv files extracted from the XDF files
output_video_folder = './video_clipped/T1/marker_MULTIPLEpairs'     # this folder will contain the clipped videos
output_audiovideo_folder = './audiovideo_sync_clipped/T1/marker_MULTIPLEpairs'     # this folder will contain the audio files extracted from the videos

print("Input video folder =", os.path.abspath(input_video_folder))
print("Input file folder =", os.path.abspath(input_file_folder))
print("Output_video folder =", os.path.abspath(output_video_folder))
print("Output_audiovideo folder =", os.path.abspath(output_audiovideo_folder))


# ------------ VARIABLES -----------------------------------------------------
# FILE-VIDEO MATCHING: List of each participant csv file e.g., ((e.g., xx_xx_Video_P1.csv) and the corresponding raw video (e.g., xx_Video_P1.avi)
participants_file_video_mapping = [
    ('Video_P1', 'P1'),
    ('Video_P2', 'P2')  # Add more participants file-video mappings as needed      
]


# AUDIO-VIDEO MATCHING: Regular expression pattern to match the participant's video file name. Change as needed. 
audio_video_pattern_matching = re.compile(r'.(P\d+)_(Vision|NoVision)_(Movement|NoMovement)_\d+_StartParticipantSinging.*(Vision|NoVision)_(Movement|NoMovement)_\d+_EndParticipantSinging')

"""In our case the files are named as [...]_PX_XVision_XMovement_X_StartParticipantSinging_XVision_XMovement_X_EndParticipantSinging[...]
        where X are changing expression inside this regular pattern. 
        The .* at the beginning and end allow for any (or no) prefix and sufffix 
    The regular expression is used to match the file names between audios and videos audiio audio-video alignment"""



# Dictionary to map file extensions to FourCC codecs
extension_to_codec = {
    '.mp4': 'H264',  # 'H264' is a valid FourCC code for H.264
    '.avi': 'XVID',  # 'XVID' is a common FourCC for AVI
    '.mov': 'H264',  # Again, use 'H264' or 'MP4V'
    '.mkv': 'H264',  # Use 'H264' for MKV files too
    '.flv': 'FLV1',  # 'FLV1' is a valid codec for FLV
    # Add more mappings as needed
}


# ------------ FUNCTIONS -----------------------------------------------------
# VIDEO: Creating a function named frame_to_time to convert frame number to time format 
def frame_to_time(frame, fps):
    """
    frame_to_time converts a given frame number to a time format (HH:MM:SS.SS) based on the frames per second (fps).
    Arguments:
        frame (int): The frame number to be converted.
        fps (float): The frames per second of the video.
    Returns:
        str: The time format as a string in the format "HH:MM:SS.SS".
    """
    seconds = frame / fps
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:.2f}"

print("Function \"frame_to_time\" created sucesfully") 



# Function to extract specified events (with correspodning LSL times) from XDF stream (useful for plotting)
def get_events(stream, event_names):
    """
    Extracts events and corresponding LSL times from the given stream that match any of the event_names.

    Parameters:
    stream (dict): The stream containing time stamps and event data.
    event_names (list of str): List of event name substrings to look for in the events.

    Returns:
    np.array: An array where each row contains a timestamp and the full event name.
    """
    events = []  # Initialize an empty list to store matching events

    # Check if the stream type is "Markers"
    if stream['info']['type'][0] != "Markers":
        raise ValueError(f"ERROR: The stream provided ({stream['info']['name'][0]}) is not a Marker stream")

    # Iterate over the time stamps and corresponding events in the stream
    for timestamp, event in zip(stream['time_stamps'], stream['time_series']):
        # Check if any of the specified event names are in the current event
        for name in event_names:
            if name in event[0]:
                # If a match is found, append the timestamp and full event name to the list
                events.append([timestamp, event[0]])

    # Convert the list of events to a NumPy array and return it
    return np.array(events)

print("Function \"get_events\" created sucesfully") 

Input video folder = c:\Users\ahmar\OneDrive\Documents\GitHub\Mobile-Multimodal-Lab\2_PREPROCESSING\1_XDF_PROCESSING\video_raw
Input file folder = c:\Users\ahmar\OneDrive\Documents\GitHub\Mobile-Multimodal-Lab\2_PREPROCESSING\1_XDF_PROCESSING\data_processed\T1\marker_MULTIPLEpairs
Output_video folder = c:\Users\ahmar\OneDrive\Documents\GitHub\Mobile-Multimodal-Lab\2_PREPROCESSING\1_XDF_PROCESSING\video_clipped\T1\marker_MULTIPLEpairs
Output_audiovideo folder = c:\Users\ahmar\OneDrive\Documents\GitHub\Mobile-Multimodal-Lab\2_PREPROCESSING\1_XDF_PROCESSING\audiovideo_sync_clipped\T1\marker_MULTIPLEpairs
Function "frame_to_time" created sucesfully
Function "get_events" created sucesfully


## 2. Clipping Videos Based on .CSV frames and LSL times

In [ ]:
# Loading the relevant CSV files that contain the LSL_time stamps and corresponding frame numbers
for participant_frame, participant_video in participants_file_video_mapping:
    
    # Loop through the files in the input_file_folder adn Loadt the file_input_folder and selecting the (all) fles containing the participant_frame name
    for file in os.listdir(os.path.abspath(input_file_folder)):
        
        if participant_frame in file and file.endswith ('csv'):   # Check if the participant_frame name is in the file name (and the file is a CSV file)
            
            print(f'Processing {participant_frame} for {participant_video}: {file}')
            
             # Load the CSV file using pandas
            file_path = os.path.join(os.path.abspath(input_file_folder), file)  # Create the file path for the CSV file
            file_data = pd.read_csv(file_path)
            
            # Find the start and end frame numbers in the CSV file 
            start_frame = file_data.iloc[0, 1]    # Get the first frame (first row, index 0) in the second coluimn (index 1)
            end_frame   = file_data.iloc[-1, 1]   # Get the last frame (last row, index -1) in the second coluimn (index 1)
            
            fnam = os.path.basename(file)[:-4]        # Extract the file name from the path and assings it to fnam, whilst removing the '.csv' extension (i.e., the last 4 characters in the string)
            
            # Find the frame rate of the LSL frames (i.e., the number of frames per second)
            LSL_frame_rate = (end_frame - start_frame) / (file_data.iloc[-1, 0] - file_data.iloc[0, 0])  # Calculate the frame rate by dividing the number of frames by the time difference between the first and last frame
            
            # Load the corresponding raw video file using OpenCV
            for video in os.listdir(os.path.abspath(input_video_folder)):
                
                if participant_video in video:  # Check if the participant_video name is in the video name
                    
                    print(f'Now Loading the Video :  {video}')
                    
                    video_path = os.path.join(os.path.abspath(input_video_folder), video)
                    
                    video_extension = os.path.splitext(video_path)[1].lower()  # Extract the file extension from the video file path, ensuring it's in lowercase (e.g., '.mp4'), to enable case-insensitive matching in the dictionary
                    
                    capture = cv2.VideoCapture(video_path) # Load the video using OpenCV
                    
                    # Extract the relevant metadata from the video
                    video_frame_width  = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))  
                    video_frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))  
                    video_frame_rate   = capture.get(cv2.CAP_PROP_FPS)
                    video_tot_frames   = int(capture.get(cv2.CAP_PROP_FRAME_COUNT)) 
                    
                    # Check video extension format and select the appropriate codec in the dictionary
                    if video_extension in extension_to_codec:
                        codec = extension_to_codec[video_extension]
                    else:
                        raise ValueError(f"ERROR: The video extension {video_extension} is not supported")
                    
                    # Assign video extension to Four Character Code (fourcc) for the codec
                    fourcc = cv2.VideoWriter_fourcc(*codec)
                    
                    # Create the output subfolder for the clipped videos
                    subfolder_path = os.path.join(os.path.abspath(output_video_folder), f'{fnam}_clipped.{video_extension[1:]}')  # Create the output subfolder for the clipped video with the correct extension
                    
                    # Initialize the VideoWriter object to write frames to a new video file based on the fourcc codec, LSL frame rate, video frame size, and subfolder path
                    out = cv2.VideoWriter(subfolder_path, fourcc, LSL_frame_rate, (video_frame_width, video_frame_height))  
                    
                    
                    # Write the selected frames to the a new clipped video 
                    capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)  # Set the video capture object to the start frame
                    with tqdm(total= end_frame - start_frame + 1,   desc="Rewriting Video Progress", leave=False, ncols=100) as pbar:    # Create a progress bar for the video frames
                        
                        frame_count = start_frame  # Initialize the frame count to the start frame
                        
                        while capture.isOpened() and frame_count <= end_frame:  # Loop through the video frames from the start frame until the end frame is reached
            
                            try: 
                                ret, frame = capture.read()   # Read the next frame from the video
                                if ret:   # If the frame is read correctly
                                    frame_count += 1
                                    pbar.update(1)
                                    out.write(frame)  # Write the frame to the new video
                                else:
                                    break
                                
                            except Exception as e:  # Catch any errors that occur during the frame processing
                                print(f"An error occurred at frame {frame_count}: {e}")
                                break  # Stop processing if an error occurs
                            
                            
                    # Release the video capture and video writer objects
                    capture.release()
                    out.release()
                        
                    print(f'Clipped video saved to: {subfolder_path}')
                        
                    print("\n")
                    
                    
print("All videos have been clipped successfully. Look into your folder " + output_video_folder) #as terminal            # Create the output subfolder for the clipped videos 

## 3. Aligning (Clipped) Vidoes with (Clipped) Audios 

In [41]:
input_video_folder = './video_clipped/T1/marker_MULTIPLEpairs'     # this folder contains the clipped videos

# LOADING AUDIOS: Loop over files in the input_file_folder to extract the relevant audio files
for audio in os.listdir(os.path.abspath(input_file_folder)):
        
    if 'Mic' in audio and 'denoised' in audio and audio.endswith('wav'):  # Correct check for 'Mic' and 'denoised'
        
        print(f'Processing the audio file:  {audio}')
        
        audio_path = os.path.join(os.path.abspath(input_file_folder), audio)
        
        # Extract the identifier from the audio file name that will be used to match the video file
        audio_match = audio_video_pattern_matching.search(audio)
        if not audio_match:
            # print(f'No matching for audio: {audio}')
            continue  # Skip current iteration if the pattern is not found
        
        audio_identifier = audio_match.group(0)  # Extract the matched portion of the string from the regex result.
        #print(f'Audio identifier: {audio_identifier}')  # Print the matched identifier for audio


        # LOADING VIDEOS: Loop over video files to select the corresponding video file
        for video in os.listdir(os.path.abspath(input_video_folder)):
            
            # Extract the identifier from the video file name that will be used to match the audio file
            video_match = audio_video_pattern_matching.search(video)
            if not video_match:
                #print(f'No matching for video: {video}')
                continue  # Skip current iteration if the pattern is not found
            
            video_identifier = video_match.group(0)  # Extract the matched portion of the string from the regex result.
            #print(f'Video identifier: {video_identifier}')  # Print the matched identifier for video

     
            # Check if the identifiers from the audio and video files match
            if audio_identifier == video_identifier: 
                
                video_path = os.path.join(os.path.abspath(input_video_folder), video)
                video_extension = os.path.splitext(video_path)[1].lower()  # Ensure correct video extension
                
                print(f'Found matching video: {video} for audio: {audio}')
                
                # Create output folder if it doesn't exist
                os.makedirs(output_audiovideo_folder, exist_ok=True)
                
                # ALIGNMENT: Combining Audio and Video using ffmpeg 
                subfolder_path = os.path.join(os.path.abspath(output_audiovideo_folder), f'{video_identifier}_audiovideo_synced.{video_extension[1:]}')
                
                # Construct the ffmpeg command to combine the audio and video files
                ffmpeg_command = [
                    'ffmpeg',              # Call the ffmpeg tool
                    '-y',                  # Overwrite the output file without asking
                    '-i', video_path,      # Specify the input video file
                    '-i', audio_path,      # Specify the input audio file
                    '-c:v', 'copy',        # Copy the video stream without re-encoding
                    '-c:a', 'aac',         # Re-encode the audio stream to AAC format (optimized for playback devices)
                    '-strict', 'experimental',  # Enable experimental features (needed for AAC encoding in some versions)
                    subfolder_path         # Specify the output file path (video with audio)
                ]
                
                # Run the command 
                print('Now combining this Audio and Video')
                try:
                    result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
                    #print(result.stdout)
                    
                    print(f'Video with audio saved as {subfolder_path}')
                    
                except subprocess.CalledProcessError as e:
                    print(f"Error combining audio and video {video_path} and {audio_path}: {e.stderr}")
                     
                
print(f'Done, you can now look into the folder: {output_audiovideo_folder}')


Processing the audio file:  T1_experiment_Mic_P1_NoVision_Movement_0_StartParticipantSinging_NoVision_Movement_0_EndParticipantSinging_denoised.wav
Found matching video: T1_experiment_Video_P1_NoVision_Movement_0_StartParticipantSinging_NoVision_Movement_0_EndParticipantSinging_clipped.avi for audio: T1_experiment_Mic_P1_NoVision_Movement_0_StartParticipantSinging_NoVision_Movement_0_EndParticipantSinging_denoised.wav
Now combining this Audio and Video
Video with audio saved as c:\Users\ahmar\OneDrive\Documents\GitHub\Mobile-Multimodal-Lab\2_PREPROCESSING\1_XDF_PROCESSING\audiovideo_sync_clipped\T1\marker_MULTIPLEpairs\_P1_NoVision_Movement_0_StartParticipantSinging_NoVision_Movement_0_EndParticipantSinging_audiovideo_synced.avi
Processing the audio file:  T1_experiment_Mic_P1_NoVision_Movement_1_StartParticipantSinging_NoVision_Movement_1_EndParticipantSinging_denoised.wav
Found matching video: T1_experiment_Video_P1_NoVision_Movement_1_StartParticipantSinging_NoVision_Movement_1_EndP